In [4]:
import torch, timm
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

model_name = "repvit_m1_0"  # 或 "repvit_m1" / "repvit_m2_3" / "repvit_m3" 等
model = timm.create_model(model_name, pretrained=True).eval()
model.eval()

# 准备预处理（自动匹配模型输入尺寸/均值方差等）
config = resolve_data_config({}, model=model)
transform = create_transform(**config)

img = Image.open("home/000/000.png").convert("RGB")
x = transform(img).unsqueeze(0)   # [1,3,H,W]

with torch.no_grad():
    logits = model(x)             # 分类 logits
    pred = logits.argmax(dim=1).item()
print("pred class id:", pred)


pred class id: 799


In [7]:
import time
import torch, timm
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import torch.nn.functional as F

# -------- device 选择（CUDA > MPS > CPU）---------
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("device:", device)

torch.backends.cudnn.benchmark = True  # 有助于加速卷积

# -------- 模型 & 预处理 ----------
model_name = "repvit_m1_0"
model = timm.create_model(model_name, pretrained=True).eval().to(device)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

# 注意：路径前面少了 "/" 的话可能找不到文件；改成你的真实路径
img = Image.open("home/000/001.png").convert("RGB")
x = transform(img).unsqueeze(0).to(device)

# -------- 预热（避免首轮 JIT / cuDNN 选择影响）---------
warmup = 10
with torch.inference_mode():
    for _ in range(warmup):
        _ = model(x)
        if device.type == "cuda":
            torch.cuda.synchronize()

# -------- 正式计时 ----------
runs = 50
start = time.perf_counter()
with torch.inference_mode():
    for _ in range(runs):
        logits = model(x)
        if device.type == "cuda":
            torch.cuda.synchronize()
end = time.perf_counter()

# 取最后一次结果做个pred展示
pred = int(logits.argmax(dim=1))
avg_ms = (end - start) * 1000.0 / runs
fps = 1000.0 / avg_ms

print(f"pred class id: {pred}")
print(f"input size: {tuple(x.shape)}")
print(f"avg latency: {avg_ms:.3f} ms  |  FPS: {fps:.2f}")


device: cpu
pred class id: 904
input size: (1, 3, 224, 224)
avg latency: 54.613 ms  |  FPS: 18.31


In [1]:
import torch
import timm

model_name = "repvit_m1_0"   # 或其他如 repvit_m1_5 / repvit_m2_3
model = timm.create_model(model_name, pretrained=True).eval()


# 如果只想保存权重参数（推荐）
torch.save(model.state_dict(), f"{model_name}_weights_only.pth")
print(f"Saved weights only to {model_name}_weights_only.pth")


/home/ranxk/miniconda3/envs/repvit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Saved weights only to repvit_m1_0_weights_only.pth


In [1]:
import timm, torch
model = timm.create_model("repvit_m1_0", pretrained=True).eval()
x = torch.zeros(1, 3, 224, 224)
y = model(x)                 # 分类输出
print("logits dim =", y.shape[1])   # 通常是 1000

# 如果想看倒数特征（forward_features + GAP）
with torch.no_grad():
    feats = model.forward_features(x)
    if feats.ndim == 4: feats = feats.mean(dim=(2,3))
print("features dim =", feats.shape[1])  # 这是特征维度（与 num_features 一致）

/home/ranxk/miniconda3/envs/repvit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


logits dim = 1000
features dim = 448
